<a href="https://colab.research.google.com/github/cwoonb/RCNN/blob/main/RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [6]:
!ls ./dataset

val2017


In [ ]:
# cho_woonbae 2021.08.09
# Do not copy without permission

In [ ]:
# Data load
import zipfile
with zipfile.ZipFile('./val2017.zip') as data_zip:
    data_zip.extractall('./data')
with zipfile.ZipFile('./panoptic_annotations_trainval2017.zip') as data_zip:
    data_zip.extractall('./annotation')

In [6]:
# Parameters

ano_dir = './annotation/annotations/panoptic_val2017.json'
root_dir = './data/val2017/'

In [6]:
import pandas as pd
import json
import numpy as np
def get_items(ano_dir):
    with open(ano_dir, 'r') as f:
        temp = json.loads(''.join(f.readlines()))
    image_list = []
    ctg_df = pd.DataFrame(temp['categories']).reset_index()
    ctg_df['index'] = ctg_df['index'] + np.ones(len(ctg_df), dtype=np.int64)
    id2ctg = dict(ctg_df.set_index('index')['id'])
    ctg2id = dict(ctg_df.set_index('id')['index'])
    for a in temp['annotations']:
        image_id = a['file_name'][:-4]
        bbox = np.stack(marking['bbox'])
        labels = np.asarray([ctg2id[l] for l in marking['category_id']])
        image_list.append({'image_id':image_id, 'bbox':bbox, 'labels':labels})
    return np.asarray(image_list), id2ctg

image_list, id2ctg = get_items(ano_dir)

In [6]:
# train_valid_split
from random import sample
def get_tv_idx(tl, k = 0.8):
    total_idx = range(tl)
    train_idx = sample(total_idx, int(tl*0.8))
    valid_idx = set(total_idx) - set(train_idx)
    return train_idx, list(valid_idx)

train_idx, valid_idx = get_tv_idx(len(image_list))

In [6]:
train_list = image_list[train_idx]
valid_list = image_list[valid_idx]

In [6]:
def get_iou(bb1, bb2):
    assert bb1[0] < bb1[1]
    assert bb1[2] < bb1[3]
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
    x_left = max(bb1[0], bb2['x1'])
    y_top = max(bb1[2], bb2['y1'])
    x_right = min(bb1[1], bb2['x2'])
    y_bottom = min(bb1[3], bb2['y2'])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1[1] - bb1[0]) * (bb1[3] - bb1[2])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

    # Selective Search로 추출한 이미지에 배경인지(0) Object인지 (1~133) 판단하기 위해 IoU를 계산해 겹치는 부분을 확인합니다.
    

In [6]:
import cv2
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

def SelectiveSearch(t):
    train_images=[]
    train_labels=[]

    img_id = t['image_id']
    img = cv2.imread(f'{root_dir}{img_id}.jpg')

    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    # print(len(ssresults))

    imout = img.copy()
    counter = 0
    falsecounter = 0
    flag = 0
    fflag = 0
    bflag = 0
    boxes = t['bbox']
    x2 = boxes[:,0] + boxes[:,2]
    y2 = boxes[:,1] + boxes[:,3]
    boxes = np.c_[boxes, x2, y2][:,[0,4,1,5]]
    for _,result in enumerate(ssresults):
        if _ < 2000 and flag == 0:
            for i, gtval in enumerate(boxes):
                x,y,w,h = result
                iou = get_iou(gtval, {"x1":x,"x2":x+w,"y1":y,"y2":y+h})
                if counter < 30:
                  # 0.7 보다 클때 이값은 iourㅏ 오브젝트로 판단
                    if iou > 0.70:
                        timage = imout[y:y+h,x:x+w]
                        resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                        train_images.append(resized)
                        train_labels.append(t['labels'][i])
                        counter += 1
                else :
                    fflag =1
                if falsecounter <30:
                                    # 0.3보다 작을때는 배경으로 레이블링 한다. 
                                    
                    if iou < 0.3:
                        timage = imout[y:y+h,x:x+w]
                        resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                        train_images.append(resized)
                        train_labels.append(0)
                        falsecounter += 1
                else :
                    bflag = 1
            if fflag == 1 and bflag == 1:
                # print("inside")
                flag = 1

    return np.array(train_images, dtype=np.uint8), np.array(train_labels, dtype=np.int_)

In [6]:
model = SEMobileNet(num_classes=10) # se net and mobile net을 합친것이다. 
model.load_state_dict(torch.load('/content/drive/MyDrive/Study_AI/semob_10class.pth'))
# 프리 트레인 모델을 가져오고 파라미터를 가져온다. 

for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(nn.Linear(1024, 4096),
                    nn.Linear(4096, len(id2ctg)+1)) 
model.cuda()

In [6]:
import torch.optim as optim
# Linear SVM Loss MSE
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(),lr=0.01, momentum=0.9, weight_decay=0.0005)

In [6]:
from torchvision import transforms
train_transform = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.RandomVerticalFlip(p=0.5),
                                      transforms.RandomHorizontalFlip(p=0.5),
])

In [ ]:
# torch형태의 모델이다. 
from tqdm.notebook import tqdm
total_loss = 0.0
tk0 = tqdm(train_list, total=len(train_list), leave=False)
# 4000장 * 60 = 12000
for idx, t in enumerate(tk0, 0):
    image_data, label_data = SelectiveSearch(t)
    inputs = torch.cat(tuple(train_transform(id).cuda().reshape(-1, 3, 224, 224) for id in image_data))
    labels = torch.Tensor(label_data).cuda()
    labels = labels.type(torch.long)

    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
